Here is the base code for takings paragraphes of dpef and splitting them into sentences, then keep only long sentences.

MIN_NB_OF_TOKENS=8 seems to do the trick. 

Improvements:
- Remove last filter and look at what has <8 words. Mostly fragments, mainly titles, etc. but may shows errors in parsing.
- Questions were kept here, maybe to exclude if appear in top sentences. but seem not frequent.
- main mistakes are of shape "['13 21 14 7 25 24 17 15 4 22 23']" --> add criteria to require actual words ?

In [53]:
import pandas as pd; pd.set_option('display.max_colwidth', -1)
# text
from spacy.lang.fr import French
from spacy.cli.download import download
from spacy.tokens import Doc, Span

# download('fr_core_news_sm')

In [54]:
input_filename = "../../data/processed/DPEFs/dpef_paragraphs_debug.csv"
output_filename = "../../data/processed/DPEFs/dpef_paragraphs_sentences_debug.csv"
output_filename2 = "../../data/processed/DPEFs/dpef_paragraphs_sentences_long_format_debug.csv"
df = pd.read_csv(input_filename, sep=";")
df = df[df.paragraph.notna()] # nan created while saving/ removing header -> to correct
df = df.loc[5000:10000]

In [58]:
# df.head()

In [65]:
# params
MIN_NB_OF_TOKENS = 3
def get_nb_words(doc):
    """ Count numer of tokens in spacy Doc, ignoring NUM and ADP (e.g. for, at...) and not counting % as noun. """
    return len([token for token in doc if (token.pos_ in ["NOUN","PROPN","VERB"]) and (token.text!="%")])

Span.set_extension("nb_words", setter=get_nb_words, getter=get_nb_words, force=True)
# custom parser to split sentences while ignoring title section like cf. and splitting on "etc."
nlp =  spacy.load('fr_core_news_sm')

def exception_to_split(token):
    if 'cf' in token.nbor(-2).text and token.nbor(-1).text == ".":
        return True
    return False

def custom_sentence_boundaries(doc):
    for i, token in enumerate(doc[2:]):
        if exception_to_split(token):
            token.is_sent_start = False
#         if exception_to_not_split(token):
#             token.is_sent_start = True
    return doc

# def exception_to_not_split(token):
#     if 'etc' in token.nbor(-2).text and token.nbor(-1).text == ".":
#         return True
#     return False

nlp.add_pipe(nlp.create_pipe('sentencizer')) # to add default sentencizer
nlp.add_pipe(custom_sentence_boundaries) # add exception to sententizer

df_sent = df["paragraph"].apply(lambda x: [sent.text for sent in nlp(x).sents if sent._.nb_words>MIN_NB_OF_TOKENS])
# save
df["paragraph_sentences"] = df_sent.values
print(df.shape)
df = df[df["paragraph_sentences"].apply(lambda x : len(x)>0)]
print(df.shape)
# df.to_csv(output_filename, sep=";") # TODO: remove

(1064, 8)
(1021, 8)


In [63]:
# reshaping following https://stackoverflow.com/questions/53860398/pandas-dataframe-how-do-i-split-one-row-into-multiple-rows-by-multi-value-colum/53860543
# convert list of pd.Series then stack it
df2 = (df
 .set_index(df.columns[:-1].values.tolist())['paragraph_sentences'] # all except last colname as index
 .apply(pd.Series)
 .stack()
 .reset_index()
 .drop('level_{}'.format(len(df.columns)-1), axis=1)
 .rename(columns={0:'sentence'}))
df2.to_csv(output_filename2, sep=";")

In [16]:
df.paragraph

5000    ● l’animation d’un Réseau de Correspondants Conformité par Business Units (BU), pays et directions centrales (DC), et une communication régulière avec les opérationnels ;                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
5001    ● le développement de la culture éthique et conformité des sal